In [169]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine

In [170]:
os.chdir('./')

# Review nutrient table and get only important values for us

In [316]:
nut = pd.read_csv("./Health_Nutritions _databases/nutrient.csv", usecols=['id', 'name', 'unit_name'],
                 index_col='id')
nut

,name,unit_name
id,,
1002,Nitrogen,G
1003,Protein,G
1004,Total lipid (fat),G
1005,"Carbohydrate, by difference",G
1007,Ash,G
...,...,...
2029,trans-Lycopene,UG
2032,"Cryptoxanthin, alpha",UG
2033,Total dietary fiber (AOAC 2011.25),G


In [321]:
importnut = nut.query("name in ['Protein', 'Energy', 'Carbohydrate, by summation', 'Sugars, Total NLEA', 'Fatty acids, total saturated', 'Fatty acids, total monounsaturated', 'Fatty acids, total polyunsaturated']")
importnut

,name,unit_name
id,,
1003,Protein,G
1008,Energy,KCAL
1050,"Carbohydrate, by summation",G
1062,Energy,kJ
1063,"Sugars, Total NLEA",G
1258,"Fatty acids, total saturated",G
1292,"Fatty acids, total monounsaturated",G
1293,"Fatty acids, total polyunsaturated",G


In [333]:
nut_list = list(importnut.index)
nut_list

[1003, 1008, 1050, 1062, 1063, 1258, 1292, 1293]

# Review food nutrient table to decrease size of table

1) To above table - add column with units from nutrition table (remove nutrient_id column, roll up fdc_id column - decreasing size of the table

In [327]:
food_nut = pd.read_csv("./Health_Nutritions _databases/food_nutrient.csv", low_memory=False,
                      usecols=['fdc_id', 'nutrient_id','amount'])
food_nut.head()

,fdc_id,nutrient_id,amount
0,323505,2048,35.0
1,326196,2048,36.0
2,747447,2048,32.0
3,321611,2048,20.0
4,746769,2048,17.0


In [335]:
food_nut.query("nutrient_id in [1003, 1008, 1050, 1062, 1063, 1258, 1292, 1293]")

,fdc_id,nutrient_id,amount
203,167512,1003,5.880
205,167512,1062,1286.000
211,167512,1258,2.941
214,167512,1008,307.000
217,167513,1003,4.340
...,...,...,...
7547652,1105781,1062,256.000
7547822,1105897,1063,12.600
7547840,1105897,1003,0.150
7547842,1105897,1008,65.000


# Review food table
1) Clean up code below - upgrade searching product, focus only on basics products
2) Make a code base on iserted variable to search some product -  in the future has to be drop-down list

In [198]:
data.loc[320354]

data_type                             market_acquistion
description    Beef, eye of round roast, raw (ER37-R-5)
Name: 320354, dtype: object

In [199]:
data.where(data['data_type'] == 'foundation_food').dropna()

,data_type,description
fdc_id,,
321358,foundation_food,"Hummus, commercial"
321359,foundation_food,"Milk, reduced fat, fluid, 2% milkfat, with add..."
321360,foundation_food,"Tomatoes, grape, raw"
321505,foundation_food,"Salt, table, iodized"
321611,foundation_food,"Beans, snap, green, canned, regular pack, drai..."
...,...,...
1105430,foundation_food,"Apples, red delicious, with skin, raw"
1105547,foundation_food,"Apples, honeycrisp, with skin, raw"
1105664,foundation_food,"Apples, granny smith, with skin, raw"


In [267]:
data['data_type'].value_counts()

branded_food                498182
sub_sample_food              12405
sr_legacy_food                7793
survey_fndds_food             7083
market_acquistion             5327
sample_food                   1829
agricultural_acquisition       810
foundation_food                177
experimental_food                6
Name: data_type, dtype: int64

In [271]:
foundation_food = data.where(data['data_type'] == 'foundation_food').dropna(how='all')
foundation_food

,data_type,description
fdc_id,,
321358,foundation_food,"Hummus, commercial"
321359,foundation_food,"Milk, reduced fat, fluid, 2% milkfat, with add..."
321360,foundation_food,"Tomatoes, grape, raw"
321505,foundation_food,"Salt, table, iodized"
321611,foundation_food,"Beans, snap, green, canned, regular pack, drai..."
...,...,...
1105430,foundation_food,"Apples, red delicious, with skin, raw"
1105547,foundation_food,"Apples, honeycrisp, with skin, raw"
1105664,foundation_food,"Apples, granny smith, with skin, raw"


In [276]:
x = foundation_food.where(data['description'].str.contains('egg whole')).dropna(how='all').index

In [281]:
type(int(x))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Int64Index'

In [172]:
data['data_type'].value_counts()

branded_food                498182
sub_sample_food              12405
sr_legacy_food                7793
survey_fndds_food             7083
market_acquistion             5327
sample_food                   1829
agricultural_acquisition       810
foundation_food                177
experimental_food                6
Name: data_type, dtype: int64

In [171]:
data = pd.read_csv("./Health_Nutritions _databases/food.csv", usecols=['fdc_id', 'data_type', 'description'], 
                   index_col='fdc_id')

In [253]:
input_food = pd.read_csv("./Health_Nutritions _databases/input_food.csv")

In [265]:
input_food.where(input_food['sr_description'].str.contains('egg')).dropna(how='all')

,id,fdc_id,fdc_id_of_input_food,seq_num,amount,sr_code,sr_description,unit,portion_code,portion_description,gram_weight,retention_code,survey_flag
5581,83894.0,1099303.0,NaN,2.0,1.5,20110.0,"Noodles, egg, enriched, cooked",C,10205.0,1 cup,240.00,0.0,NaN
5607,83922.0,1099307.0,NaN,5.0,4.0,20110.0,"Noodles, egg, enriched, cooked",C,10205.0,1 cup,640.00,0.0,NaN
5616,83929.0,1099308.0,NaN,3.0,50.0,20110.0,"Noodles, egg, enriched, cooked",GM,0.0,NONE,50.00,0.0,NaN
5621,83934.0,1099309.0,NaN,3.0,50.0,20110.0,"Noodles, egg, enriched, cooked",GM,0.0,NONE,50.00,0.0,NaN
5626,83945.0,1099310.0,NaN,9.0,2.0,20110.0,"Noodles, egg, enriched, cooked",C,10205.0,1 cup,320.00,385.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21758,100073.0,1103325.0,NaN,2.0,0.5,20110.0,"Noodles, egg, enriched, cooked",C,10205.0,1 cup,80.00,0.0,NaN
21759,100076.0,1103326.0,NaN,3.0,14.0,20110.0,"Noodles, egg, enriched, cooked",GM,0.0,NONE,14.00,0.0,NaN
23041,101355.0,1103729.0,NaN,7.0,3.0,20109.0,"Noodles, egg, dry, enriched",WO,0.0,NONE,85.05,384.0,NaN
23049,101368.0,1103731.0,NaN,7.0,3.0,20109.0,"Noodles, egg, dry, enriched",WO,0.0,NONE,85.05,384.0,NaN
